In [10]:
import math
import time
from collections import deque
from queue import PriorityQueue

## Loading and preprocessing input

In [11]:
def load_data(input_location: str):
    with open(input_location, 'r') as f:
        data = f.read()
        
    if not data:
        print("Error opening file")
        
    data = data.split("\n")
    return data

In [12]:
def get_samples(data: list):
    samples = []
    lis = []
    for x in data:
        if x == '':
            if lis != []:
                idx = 0
                for line in lis:
                    line_as_list = []
                    found_start = False
                    for char in line:
                        if char == " " and not found_start:
                            line_as_list.append("#")
                        else:
                            found_start = True
                            line_as_list.append(char)
                    lis[idx] = ''.join(line_as_list)
                    idx += 1
                
                max_len = len(max(lis, key=len))
                
                idx = 0
                for line in lis:
                    pad_len = max_len - len(line)
                    padding = '#'*pad_len
                    lis[idx] += padding
                    idx += 1
                
                top_bottom_fill = '#'*max_len
                lis[0] = top_bottom_fill
                lis[len(lis)-1] = top_bottom_fill
            
                samples.append(lis)
            lis = []
            continue
        else:
            if x[0] == ';':
                continue
        lis.append(x)
    return samples

## Sokoban Game class

In [13]:
class Sokoban():
    def __init__(self, walls: set, boxes: set, storages: set, player: tuple, width: int, height: int, grid: list):
        self.walls = walls
        self.boxes = boxes
        self.storages = storages
        self.player = player
        self.cols = width # width
        self.rows = height # height
        self.grid = grid
        # distance from all positions to all goals or storages 
        # (goal) as key and assinging values to all postions for every goal
        self.distanceToGoal = dict()
        self.__init_goalPull()
        
        self.deadlocks = set()
        
    def inbound(self, x: int, y: int):
        if x >= 1 and y >= 1 and x <= self.rows and y <= self.cols:
            return True
        return False
    
    def printMap(self, state):
        self.__printGrid(self.__loadGrid(state.boxes, state.player))
    
    def __init_goalPull(self):
        self.__load_dict()
        for goal in self.storages:
            self.distanceToGoal[goal][goal] = 0
            q = deque()
            q.append((goal))
            while q:
                pos = q.popleft()
                for new_coord in self.__move_pos(pos):
                    boxPosition = new_coord[0]
                    playerPosition = new_coord[1]
                    if self.distanceToGoal[goal][boxPosition] == float("inf"):
                        if boxPosition not in self.walls and playerPosition not in self.walls:
                            self.distanceToGoal[goal][boxPosition] = self.distanceToGoal[goal][pos] + 1
                            q.append(boxPosition)
    
    def __load_dict(self):
        positions = set()
        for i in range(1, self.rows+1):
            for j in range(1, self.cols+1):
                positions.add((i, j))
                
        for goal in self.storages:
            self.distanceToGoal[goal] = dict()
            for pos in positions:
                self.distanceToGoal[goal][pos] = float("inf")
                
    def __loadGrid(self, boxes: set, player: tuple):
        grid = [[" " for _ in range(self.cols)] for _ in range(self.rows)] 
        for coord in self.walls:
            x = coord[0]-1
            y = coord[1]-1
            grid[x][y] = "#"
      
        for coord in self.storages:
            x = coord[0]-1
            y = coord[1]-1
            grid[x][y] = "."
            
        for coord in boxes:
            x = coord[0]-1
            y = coord[1]-1
            grid[x][y] = "$"
            
        for coord in boxes.intersection(self.storages):
            x = coord[0]-1
            y = coord[1]-1
            grid[x][y] = "*"
            
        coord = player
        x = coord[0]-1
        y = coord[1]-1
        
        if player in self.storages:
            grid[x][y] = "+"
        else:
            grid[x][y] = "@"
        
        return grid
            
    def __printGrid(self, grid):
        for i in range(0, self.rows):
            for j in range(0, self.cols):
                print(grid[i][j], end=" ")
            print()
                
    def __move_pos(self, pos):
        x = pos[0]
        y = pos[1]
        
        return [((x-1, y), (x-2, y)), ((x+1, y), (x+2, y)), ((x, y-1), (x, y-2)), ((x, y+1), (x, y+2))]

## Board Generator

In [14]:
def boards(samples: list):
    for example in samples:
        sizeH, sizeV = len(example[0]), len(example)
        input_list = [[sizeH, sizeV]]
        
        #########
        width, height = sizeH, sizeV
        walls, boxes, storages = set(), set(), set()
        player = None
        #########

        i = 1
        j = 1
        count_wall = 0
        count_box = 0
        count_storage_locations = 0
        nWallSquares = []
        nBoxes = []
        nStorageLocations = []
        player_coord = []
        for line in example:
            j = 1
            for char in line:
                # player and goal or storage
                if char == "+":
                    count_storage_locations += 1
                    nStorageLocations.append(i)
                    nStorageLocations.append(j)
                    storages.add((i, j))
                    player_coord = [i, j]
                    player = (i, j)
                # box and goal or storage
                if char == "*":
                    count_box += 1
                    nBoxes.append(i)
                    nBoxes.append(j)
                    boxes.add((i, j))
                    count_storage_locations += 1
                    nStorageLocations.append(i)
                    nStorageLocations.append(j)
                    storages.add((i, j))
                # wall
                if char == '#':
                    count_wall += 1
                    nWallSquares.append(i)
                    nWallSquares.append(j)
                    walls.add((i, j))
                # box
                if char == '$':
                    count_box += 1
                    nBoxes.append(i)
                    nBoxes.append(j)
                    boxes.add((i, j))
                # storage or goal
                if char == '.':
                    count_storage_locations += 1
                    nStorageLocations.append(i)
                    nStorageLocations.append(j)
                    storages.add((i, j))
                # player
                if char == '@':
                    player_coord = [i, j]
                    player = (i, j)
                j += 1
            i += 1

        nWallSquares.insert(0, count_wall)
        input_list.append(nWallSquares)
    
        nBoxes.insert(0, count_box)
        input_list.append(nBoxes)

        nStorageLocations.insert(0, count_box)
        input_list.append(nStorageLocations)

        input_list.append(player_coord)
        
        input_in_another_format = ''
        first = True
        for line in input_list:
            if first:
                input_in_another_format = ' '.join(str(v) for v in line)
                first = False
            else:
                input_in_another_format = input_in_another_format + '\n' + ' '.join(str(v) for v in line)
        
        grid = example
        
        Game = Sokoban(walls=walls, boxes=boxes, storages=storages, player=player, width=width, height=height, grid=example)
                
        yield Game

## Deadlock generator function

In [15]:
class DeadLockGenerator():
    def __init__(self, game: Sokoban):
        self.game = game
        self.deadlocks = set()
        self.game.width = self.game.cols
        self.game.height = self.game.rows
        
    def getDeadLocks(self):
        for i in range(1, self.game.rows+1):
            for j in range(1, self.game.cols+1):
                coord = (i, j)
                if coord not in self.game.walls and coord not in self.game.storages:
                    if self.cornerTest(coord) or self.boundaryTest(coord):
                        self.deadlocks.add(coord)
                        
    def cornerTest(self, coord: tuple):
        top_coord = (coord[0]-1, coord[1])
        down_coord = (coord[0]+1, coord[1])
        left_coord = (coord[0], coord[1]-1)
        right_coord = (coord[0], coord[1]+1)
        
        walls = self.game.walls
        if (top_coord in walls and right_coord in walls) or (top_coord in walls and left_coord in walls) or (down_coord in walls and left_coord in walls) or (down_coord in walls and right_coord in walls):
            return True
        return False
        
    def boundaryTest(self, coord: tuple):
        x = coord[0]
        y = coord[1]
        top_coord = (x-1, y)
        down_coord = (x+1, y)
        left_coord = (x, y-1)
        right_coord = (x, y+1)
        
        walls = self.game.walls
        
        upbound = self.__findNearestUpbound(coord)
        downbound = self.__findNearestDownbound(coord)
        rightbound = self.__findNearestRightbound(coord)
        leftbound = self.__findNearestLeftbound(coord)
            
        if left_coord in walls:
            if upbound == -1 or downbound == -1: 
                return False
            else:
                for m in range(upbound+1, downbound):
                    if (m, y-1) not in walls:
                        return False
            return True
        
        if right_coord in walls:
            if upbound == -1 or downbound == -1: 
                return False
            else:
                for m in range(upbound+1, downbound):
                    if (m, y+1) not in walls:
                        return False
            return True
        
        if top_coord in walls:
            if leftbound == -1 or rightbound == -1: 
                return False
            else:
                for m in range(leftbound+1, rightbound):
                    if (x-1, m) not in walls:
                        return False
            return True
        
        if down_coord in walls:
            if leftbound == -1 or rightbound == -1: 
                return False
            else:
                for m in range(leftbound+1, rightbound):
                    if (x+1, m) not in walls:
                        return False
            return True
        return False
    
    def __findNearestUpbound(self, coord: tuple):
        y = coord[1]
        x = coord[0] - 1
        
        while x >= 0:
            if (x, y) in self.game.walls:
                return x
            elif (x, y) in self.game.storages:
                return -1
            x -= 1
        return 0
    
    def __findNearestDownbound(self, coord: tuple):
        y = coord[1]
        x = coord[0] + 1
        
        while x <= self.game.height:
            if (x, y) in self.game.walls:
                return x
            elif (x, y) in self.game.storages:
                return -1
            x += 1
        return self.game.height
    
    def __findNearestRightbound(self, coord: tuple):
        y = coord[1] + 1
        x = coord[0]
        
        while y < self.game.width:
            if (x, y) in self.game.walls:
                return y
            elif (x, y) in self.game.storages:
                return -1
            y += 1
        return self.game.width
    
    def __findNearestLeftbound(self, coord: tuple):
        y = coord[1] - 1
        x = coord[0] 
        
        while y >= 0:
            if (x, y) in self.game.walls:
                return y
            elif (x, y) in self.game.storages:
                return -1
            y -= 1
        return 0

## State representation and Functions

In [16]:
class State():
    def __init__(self, boxes: set, player: tuple, path: str):
        self.boxes = boxes
        self.player = player
        self.path = path
    
    def getNeighbors(self, game: Sokoban):
        x = self.player[0]
        y = self.player[1]
        
        neighbours = []
        neighbours.append(self.__move(x-1, y, x-2, y, "u", game))
        neighbours.append(self.__move(x+1, y, x+2, y, "d", game))
        neighbours.append(self.__move(x, y-1, x, y-2, "l", game))
        neighbours.append(self.__move(x, y+1, x, y+2, "r", game))
        
        return [node for node in neighbours if not node == None]

    def __move(self, ax: int, ay: int, bx: int, by: int, s: str, game: Sokoban):
        attempt = (ax, ay)
        newbox = (bx, by)
        changed = False
        
        if not game.inbound(ax, ay):
            return
         
        if attempt not in game.walls:
            if attempt in self.boxes:
                if newbox in self.boxes or not game.inbound(bx, by) or newbox in game.walls or newbox in game.deadlocks:
                    return
                s = s.upper()
                self.boxes.remove(attempt)
                self.boxes.add(newbox)
                changed = True
            cur = State(boxes=set(self.boxes), player=attempt, path=self.path+s)
            if changed:
                self.boxes.remove(newbox)
                self.boxes.add(attempt)
            return cur
            
    def reachedGoal(self, game: Sokoban):
        return game.storages == self.boxes
    
    def euclidean(self, game: Sokoban):
        return self.__heuristicHelper(operation="euclidean", game=game)
    
    def manhattan(self, game: Sokoban):
        return self.__heuristicHelper(operation="manhattan", game=game)
    
    def goalpull(self, game: Sokoban):            
        boxesToStorages = 0
        for goal in game.storages:
            for box in self.boxes:
                boxesToStorages += game.distanceToGoal[goal][box]
        return boxesToStorages
    
    def minmatch(self, game: Sokoban):
        matching = self.__greedy_matching(game)
        lower_bound = 0
        for (box, goal) in matching:
            lower_bound += game.distanceToGoal[goal][box]
        return lower_bound
    
    def __greedy_matching(self, game: Sokoban):
        pq = PriorityQueue()
        count = 0
        for goal in game.storages:
            for box in self.boxes:
                count += 1
                element = (game.distanceToGoal[goal][box], count, goal, box)
                pq.put(element)
        
        matchedBoxes = set()
        matchedGoals = set()
        # matching is box coord tuple, goal coord tuple
        matching = set()
        
        while not pq.empty():
            top = pq.get()
            box = top[3]
            goal = top[2]
            if box not in matchedBoxes and goal not in matchedGoals:
                matching.add((box, goal))
                matchedBoxes.add(box)
                matchedGoals.add(goal)
        
        for box in self.boxes:
            if box not in matchedBoxes:
                goal, dist = self.__closestGoal(box, game.distanceToGoal)
                matching.add((box, goal))
        
        return matching
                
                
    def __closestGoal(self, box: tuple, distanceToGoal: dict):
        min_goal = None
        min_dist = None
        first = True
        for goal in distanceToGoal:
            temp_dist = distanceToGoal[goal][box]
            if first:
                min_dist = temp_dist
                min_goal = goal
                first = False
            else:
                if temp_dist < min_dist:
                    min_goal = goal
                    min_dist = temp_dist
        return min_goal, min_dist
    
    def __heuristicHelper(self, operation: str, game: Sokoban):
        x = self.player[0]
        y = self.player[1]
        
        playerToBoxes = 0
#         for coord in self.boxes:
#             bx = coord[0]
#             by = coord[1]
#             playerToBoxes += self.__opr(x1=x, x2=bx, y1=y, y2=by, operation=operation)
            
        boxesToStorages = 0
        for coord in game.storages:
            sx = coord[0]
            sy = coord[1]
            for box_coord in self.boxes:
                bx = box_coord[0]
                by = box_coord[1]
                boxesToStorages += self.__opr(x1=sx, x2=bx, y1=sy, y2=by, operation=operation)
                
        return playerToBoxes + boxesToStorages
    
    def __opr(self, x1: int, x2: int, y1: int, y2: int, operation: str):
        if operation == "euclidean":
            return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))
        elif operation == "manhattan":
            return abs(x1-x2) + abs(x1-x2)
    
    def __str__(self):
        return f'Current status: player at {self.player} with path {self.path}'
    
    def __hash__(self):
        boxHashCode = 0
        for e in self.boxes:
            boxHashCode += hash(e)
            boxHashCode *= 37
        return self.player[0]*73 + self.player[1] + boxHashCode
    
    def __eq__(self, other):
        if not isinstance(other, type(self)): return NotImplemented
        return self.boxes == other.boxes and self.player == other.player

## Search

In [29]:
def f_n(state: State, heuristic: str, game: Sokoban, algo: str = None, count: int = 0):
    h = 0
    if algo == "astar":
        h = count
    if heuristic == "euclidean":
        return state.euclidean(game) + h
    if heuristic == "manhattan":
        return state.manhatten(game) + h
    if heuristic == "minmatch":
        return state.minmatch(game) + h
    if heuristic == "goalpull":
        return state.goalpull(game) + h
        
def informed(state: State, heuristic: str, algo: str, game: Sokoban):
    start = time.time()
    
    notVisited = PriorityQueue()
    visited = set()
    
    count = 1
    element = (f_n(state, heuristic, game, algo, count), count, state)
    notVisited.put(element)
    
    nodes_expanded = 0
    
    while not notVisited.empty():
        top = notVisited.get()
        element = top[2]
        visited.add(element)
        if element.reachedGoal(game):
            print(f'Path: {element.path}')
            game.printMap(element)
            print(f'{algo} {heuristic} in {(time.time() - start)*1000} ms')
            print(f'Node expanded: {nodes_expanded}')
            print(f'Length of Path: {len(element.path)}')
            print()
            with open("output.txt", "a+") as f:
                f.seek(0)
                data = f.read(100)
                if len(data) > 0:
                    f.write("\n")
                f.write(element.path)
                f.close()
            break
        else:
            nodes_expanded += 1
            for neighbour in element.getNeighbors(game):
                if neighbour not in visited:
                    count += 1
                    neighbour_element = (f_n(element, heuristic, game, algo, count), count, neighbour)
                    notVisited.put(neighbour_element)
                    
def idastar(state: State, heuristic: str, game: Sokoban):
    threshold = f_n(state=state, heuristic=heuristic, game=game)
    while True:
        temp = idastar_search(state, heuristic, game, distance=0, threshold=threshold)
        if temp == "Found":
            return "Found"
        if temp == float("inf"):
            return
        threshold = temp
            
        
def idastar_search(state: State, heuristic: str, game: Sokoban, distance, threshold: float):
    if state.reachedGoal():
        state.printMap()
        return "Found"
    estimate = distance + f_n(state=state, heuristic=heuristic, game=game)
    if estimate > threshold:
        return estimate
    
    minimum = float("inf")
    for neighbour in state.getNeighbors(game):
        temp = idastar_search(state=neighbour, heuristic=heuristic, game=game, distance=distance+f_n(state=neighbour, heuristic=heuristic, game=game), threshold=threshold)
        if temp == "Found":
            return temp
        if temp < minimum:
            minimum = temp
    return minimum

## Running the code and getting output

In [26]:
data = load_data('testExamples.xsb')
samples = get_samples(data)

In [27]:
for Game in boards(samples):
    for line in Game.grid:
        print(line)
    print()

#########
#...  . #
######  #
#     $##
# $.   ##
# # #####
#@$$ $ ##
#      ##
#########

#########
#@  #  ##
# ..   ##
##.##   #
##  $$# #
####$   #
####  ###
#########

###########
#### # ####
###  ###  #
## $      #
#   @$ #  #
### $###  #
###  #..  #
### ##.# ##
##      ###
##     ####
###########

##############
##############
##       #####
## ##### #####
## #   # #####
## $.#@# #####
###$.  # #####
#  $.### #####
# #$.    #####
#    #########
##############

###########
##  ##  ###
##.$   .###
## .## $  #
##$### #  #
# $    # ##
# @ #.    #
#######   #
###########

####################
#####   #          #
#####$  #          #
#####  $##         #
###  $ $ #         #
### # ## #   #######
#   # ## #####  ..##
# $  $          ..##
##### ### #@##  ..##
#####     ##########
####################



In [28]:
for Game in boards(samples):
    x = DeadLockGenerator(Game)
    x.getDeadLocks()
    Game.deadlocks = x.deadlocks
    start = State(boxes=set(Game.boxes), player=Game.player, path="")
    informed(state=start, heuristic="minmatch", algo="greedy", game=Game)
#     idastar(state=start, heuristic="minmatch", distanceToGoal=Game.distanceToGoal)

Path: uuRurrdrrUUdlllldldddrrUdlluRluuurrrrruruLLLLLrrrrddlllldldddrrruLdrrruLdlllluuurRlldddrruUUluRRRdLrrUUdlllldlddRdrUdrruLdlluluuurrrrruruLLLLrrrddlllldRllddrRUUluRRRdLrrUUdlllldlddrruUluRRdldddrruLdlUUdlluuurrrdrruuruLLLrrdddlluRdrUU
# # # # # # # # # 
# * * *     *   # 
# # # # # # @   # 
#             # # 
#     *       # # 
#   #   # # # # # 
#             # # 
#             # # 
# # # # # # # # # 
greedy minmatch in 1068.0956840515137 ms
Node expanded: 6189
Length of Path: 233

Path: drrrrdrrddllUUruLLLulldRddrRlluuRRRdrrddllUllluurrrurDllllddrrrUdddlUrrruuLuLLLrrDDuullulldRddrRdRUUruulDlLulDrrrDDlddrUUUruLLLrrdrrddLruulldlddrUUUruLL
# # # # # # # # # 
#       #     # # 
#   * * @     # # 
# # * # #       # 
# #         #   # 
# # # #         # 
# # # #     # # # 
# # # # # # # # # 
greedy minmatch in 533.8256359100342 ms
Node expanded: 4520
Length of Path: 152

Path: uulDDDDDurUUluurDrrrrdddlldddlllluRRluuurUluRRRRldLLulDrddldddrrruuurruuruulDDulldlllDDDuuuurrrrrdrddllldddll

## References:
- Goal pull distance and minmatch algorithm were coded based on pseudocode from https://baldur.iti.kit.edu/theses/SokobanPortfolio.pdf
- The ideas for deadlock were referenced fom https://github.com/dabaitudiu/SokobanAI/blob/master/SokobanAI.pdf

## Team
Kovvuri Sai Gopal Reddy - 2010110354 

S.Gokul Sharan - 2010110543

Mahammad Siraj Cheruvu - 2010110378

Bhanu Sathwik Nullu - 2010110443

Posam Yeswath Reddy - 2010110466